In [3]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt

In [4]:
dataframe = pd.read_excel(r'G:\traindata\RFMtest\tmp.xlsx')
dataframe.head()

,payment,trdtime,id,name
0,1.0,2019-12-25 01:07:14,13,吕游
1,NaN,2019-12-30 07:39:22,13,吕游
2,48.5,2020-06-22 11:51:51,46,卢芬
3,16.5,2020-06-27 02:32:09,46,卢芬
4,221.4,2020-06-30 12:15:34,46,卢芬


In [5]:
dataframe['trdtime'] = pd.to_datetime(dataframe['trdtime'])
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   payment  390 non-null    float64       
 1   trdtime  394 non-null    datetime64[ns]
 2   id       394 non-null    int64         
 3   name     393 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 12.4+ KB


In [6]:
dataframe['trdtime'].min()

Timestamp('2019-12-25 01:07:14')

In [13]:
dataframe['trdtime'].max()

Timestamp('2020-08-25 04:45:54')

In [15]:
R_today_1 = dataframe['trdtime'].max()+ pd.DateOffset(days=1)
R_today_1

Timestamp('2020-08-26 04:45:54')

In [122]:
R_today = dt.datetime(2020,8,27)

In [123]:
dataframe['R_diff'] = (R_today-dataframe['trdtime']).dt.days
dataframe.head()

,payment,trdtime,id,name,R_diff
0,1.0,2019-12-25 01:07:14,13,吕游,245
1,NaN,2019-12-30 07:39:22,13,吕游,240
2,48.5,2020-06-22 11:51:51,46,卢芬,65
3,16.5,2020-06-27 02:32:09,46,卢芬,60
4,221.4,2020-06-30 12:15:34,46,卢芬,57


In [124]:
R = dataframe.groupby(by = ['id'])['R_diff']
R.head()

0      245
1      240
2       65
3       60
4       57
      ... 
389      2
390      2
391      2
392      2
393      1
Name: R_diff, Length: 373, dtype: int64

In [125]:
R = R.agg([('R_diff','min')])

,R_diff
id,
13,240
46,38
48,13
51,18
82,239


In [126]:
F = dataframe.groupby(by = ['id'])['id'].agg([('F_fre','count')])

In [127]:
M = dataframe.groupby(by = ['id'])['payment'].agg([('M_sum',sum)])
RFM = R.join(F).join(M)
RFM.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262 entries, 13 to 514
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   R_diff  262 non-null    int64  
 1   F_fre   262 non-null    int64  
 2   M_sum   262 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 18.2 KB


In [128]:
RFM.head()

,R_diff,F_fre,M_sum
id,,,
13,240,2,1.0
46,38,5,347.4
48,13,4,170.6
51,18,7,606.3
82,239,4,1.0


In [129]:
RFM["R"]=np.where(RFM.R_diff<90,5,np.where(RFM.R_diff<=180,4,np.where(RFM.R_diff<=270,3,np.where(RFM.R_diff<=360,2,1))))

RFM["F"]=np.where(RFM.F_fre==1,1,np.where(RFM.F_fre==2,2,np.where(RFM.F_fre==3,3,np.where(RFM.F_fre==4,4,5))))
                  
RFM["M"]=np.where(RFM.M_sum<50,1,np.where(RFM.M_sum<=100,2,np.where(RFM.M_sum<=200,3,np.where(RFM.M_sum<=500,4,5))))
                  
RFM.head()

,R_diff,F_fre,M_sum,R,F,M
id,,,,,,
13,240,2,1.0,3,2,1
46,38,5,347.4,5,5,4
48,13,4,170.6,5,4,3
51,18,7,606.3,5,5,5
82,239,4,1.0,3,4,1


In [130]:
RFM.describe()

,R_diff,F_fre,M_sum,R,F,M
count,262.000000,262.000000,262.000000,262.000000,262.000000,262.00000
mean,29.015267,1.503817,72.784733,4.980916,1.423664,1.69084
std,24.710891,1.352585,97.306767,0.184708,0.926356,0.96283
min,1.000000,1.000000,1.000000,3.000000,1.000000,1.00000
25%,18.000000,1.000000,24.350000,5.000000,1.000000,1.00000
50%,26.000000,1.000000,42.950000,5.000000,1.000000,1.00000
75%,34.000000,1.000000,79.400000,5.000000,1.000000,2.00000
max,240.000000,14.000000,964.700000,5.000000,5.000000,5.00000


In [131]:
# bins = RFM['R_diff'].quantile(q = np.linspace(0,1,6),interpolation = 'nearest')
# bins[0] = 0
# labels =  [5,4,3,2,1]
# R = pd.cut(RFM['R_diff'],bins,labels = labels)

# bins = [1,2,5,8,200]
# labels = np.arange(1,5)
# F = pd.cut(RFM['F_fre'], bins=bins, labels=labels,include_lowest=True)

# bins = RFM['M_sum'].quantile(q = np.linspace(0,1,6),interpolation = 'nearest')
# bins[0] = 0
# labels = [1,2,3,4,5]
# M = pd.cut(RFM['M_sum'],bins,labels = labels)

# RFM['R'] = R
# RFM['F'] = F
# RFM['M'] = M
# RFM.head()

In [132]:
RFM.describe()

,R_diff,F_fre,M_sum,R,F,M
count,262.000000,262.000000,262.000000,262.000000,262.000000,262.00000
mean,29.015267,1.503817,72.784733,4.980916,1.423664,1.69084
std,24.710891,1.352585,97.306767,0.184708,0.926356,0.96283
min,1.000000,1.000000,1.000000,3.000000,1.000000,1.00000
25%,18.000000,1.000000,24.350000,5.000000,1.000000,1.00000
50%,26.000000,1.000000,42.950000,5.000000,1.000000,1.00000
75%,34.000000,1.000000,79.400000,5.000000,1.000000,2.00000
max,240.000000,14.000000,964.700000,5.000000,5.000000,5.00000


In [133]:
RFM_TEST = RFM.copy()

In [134]:
cluster=[]
for i in range(len(RFM_TEST)):
    if RFM_TEST.iloc[i,3] >= 3 and RFM_TEST.iloc[i,4] >=3 and RFM_TEST.iloc[i,5] >=3:
        cluster.append('重要价值客户')
    elif RFM_TEST.iloc[i,3] < 3 and RFM_TEST.iloc[i,4] >= 3 and RFM_TEST.iloc[i,5] >=3:
        cluster.append('重要保持客户')
    elif RFM_TEST.iloc[i,3] >= 3 and RFM_TEST.iloc[i,4] < 3 and RFM_TEST.iloc[i,5] >=3:
        cluster.append('重要发展客户')
    elif RFM_TEST.iloc[i,3] < 3 and RFM_TEST.iloc[i,4] < 3 and RFM_TEST.iloc[i,5] >= 3:
        cluster.append('重要挽留客户')
    elif RFM_TEST.iloc[i,3] >= 3 and RFM_TEST.iloc[i,4] >= 3 and RFM_TEST.iloc[i,5] < 3:
        cluster.append('一般价值客户')
    elif RFM_TEST.iloc[i,3] < 3 and RFM_TEST.iloc[i,4] >= 3 and RFM_TEST.iloc[i,5] < 3:
        cluster.append('一般保持客户')
    elif RFM_TEST.iloc[i,3] >= 3 and RFM_TEST.iloc[i,4] < 3 and RFM_TEST.iloc[i,5] < 3:
        cluster.append('一般发展客户')
    else:
        cluster.append('一般挽留客户')
RFM_TEST['type'] = cluster

In [135]:
RFM_TEST = RFM_TEST.reset_index()

In [136]:
RFM_TEST.head()

,id,R_diff,F_fre,M_sum,R,F,M,type
0,13,240,2,1.0,3,2,1,一般发展客户
1,46,38,5,347.4,5,5,4,重要价值客户
2,48,13,4,170.6,5,4,3,重要价值客户
3,51,18,7,606.3,5,5,5,重要价值客户
4,82,239,4,1.0,3,4,1,一般价值客户


In [1]:
result = RFM_TEST[['id','type']]
result.info()

NameError: name 'RFM_TEST' is not defined

In [137]:
result.to_excel('YD_result.xlsx')